# Deliverable 2 
#Step 1 ReCreate the vine_table DataFrame from Amazon Reviews ETL

In [1]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example: spark_version = 'spark-3.0.3'
spark_version = 'spark-3.0.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Hit:3 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:6 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Ign:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:9 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:11 http://security.ubuntu.com/ubuntu bionic-security/universe amd64 Packages [1,431 kB]
Get:12 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 

In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Vine Analysis").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [3]:
# load in a sql function to use columns for filtering
from pyspark.sql.functions import col

In [4]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Books_v1_01.tsv.gz"
spark.sparkContext.addFile(url)
books_df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)
books_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   22480053|R28HBXXO1UEVJT|0843952016|      34858117|          The Rising|           Books|          5|            0|          0|   N|                N|Great Twist on Zo...|I've known about ...| 2012-05-03|
|         US|   44244451| RZKRFS2UUMFFU|031088926X|     676347131|Sticky Faith Teen...|           Books|          5|    

In [5]:
# Create the vine_table DataFrame
vine_df = books_df.select(['review_id','star_rating','helpful_votes','total_votes','vine','verified_purchase'])
vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R28HBXXO1UEVJT|          5|            0|          0|   N|                N|
| RZKRFS2UUMFFU|          5|           15|         15|   N|                Y|
|R2WAU9MD9K6JQA|          3|            6|          8|   N|                N|
|R36SCTKYTVPZPC|          5|           10|         11|   N|                Y|
|R10BM6JUOJX27Q|          3|            0|          0|   Y|                N|
| RCLZ5OKZNUSY4|          5|            0|          0|   N|                Y|
|R1S65DJYEI89G4|          4|            8|         17|   N|                N|
|R3KQYBQOLYDETV|          4|            2|          2|   N|                N|
|R3QV8K7CSU8K2W|          5|            0|          0|   N|                N|
|R3W5A1WUGO5VQ0|          4|            0|          1|   N|     

# Step 2 Filter Vine Data for Analysis

In [6]:
# Filter the data and create a new DataFrame or table to retrieve all the rows where the total_votes count is equal to or greater than 20
totalOver20_df = vine_df.filter(col("total_votes") >= 20)
totalOver20_df.show(10)

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R18VIM840CEFRP|          1|           16|        105|   N|                N|
|R14PMAJTY0EAAT|          4|          135|        142|   N|                Y|
|R1363VA3TPNLVB|          5|          370|        388|   N|                Y|
| RBQZC5A3TSWT5|          5|           11|         22|   N|                Y|
| RW00TDPV9U93E|          1|           35|         76|   N|                N|
|R3OW0AIVLEDIQ7|          3|           99|        121|   N|                N|
|R3DTESO4FUAPKQ|          5|           90|        102|   N|                N|
|R18I0XBYWFSR5T|          1|           16|         50|   N|                N|
|R1VSQ3QK30VDI0|          5|           23|         35|   N|                N|
|R3NEJBF4FXMVUU|          5|           87|         90|   N|     

In [7]:
# Filter the new DataFrame or table created above 
# and create a new DataFrame or table to retrieve all the rows where the number of helpful_votes divided by total_votes is equal to or greater than 50%
helpful_df = totalOver20_df.filter((totalOver20_df['helpful_votes']/totalOver20_df['total_votes']) >= 0.5)
helpful_df.show(10)


+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R14PMAJTY0EAAT|          4|          135|        142|   N|                Y|
|R1363VA3TPNLVB|          5|          370|        388|   N|                Y|
| RBQZC5A3TSWT5|          5|           11|         22|   N|                Y|
|R3OW0AIVLEDIQ7|          3|           99|        121|   N|                N|
|R3DTESO4FUAPKQ|          5|           90|        102|   N|                N|
|R1VSQ3QK30VDI0|          5|           23|         35|   N|                N|
|R3NEJBF4FXMVUU|          5|           87|         90|   N|                Y|
|R34ZTQTHWRHT5R|          5|           18|         20|   N|                Y|
|R367X555006ROC|          1|           39|         44|   N|                N|
| R4NCMAZCRAXHT|          3|           19|         21|   N|     

In [8]:
# Filter the DataFrame or table created in helpful, 
# and create a new DataFrame or table that retrieves all the rows where a review was written as part of the Vine program (paid), vine == 'Y
vineYes_df = helpful_df.filter(col('vine') == 'Y')
vineYes_df.show(5)

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R2Z8083V8ZFQVZ|          3|           82|         87|   Y|                N|
| RK25TAO4GGS5G|          3|           22|         24|   Y|                N|
|R2SJQG3C6KY0M8|          3|           18|         20|   Y|                N|
|R12CHC0CB2WASU|          5|           43|         48|   Y|                N|
|R2RY328TIDXMTE|          2|           50|         61|   Y|                N|
+--------------+-----------+-------------+-----------+----+-----------------+
only showing top 5 rows



In [9]:
# Repeat, but this time retrieve all the rows where the review was not part of the Vine program (unpaid), vine == 'N'.
vineNo_df = helpful_df.filter(col('vine')== 'N')
vineNo_df.show(5)

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R14PMAJTY0EAAT|          4|          135|        142|   N|                Y|
|R1363VA3TPNLVB|          5|          370|        388|   N|                Y|
| RBQZC5A3TSWT5|          5|           11|         22|   N|                Y|
|R3OW0AIVLEDIQ7|          3|           99|        121|   N|                N|
|R3DTESO4FUAPKQ|          5|           90|        102|   N|                N|
+--------------+-----------+-------------+-----------+----+-----------------+
only showing top 5 rows



# Step 3 Vine Analysis
Determine the total number of reviews, the number of 5-star reviews and the percentage of 5-star reviews for the two types of review (paid vs unpaid)

In [10]:
# total reviews
total_reviews = helpful_df.count()
total_reviews

337176

In [11]:
# total number of unpaid reviews
unpaid_reviews = vineNo_df.count()
unpaid_reviews

332395

In [12]:
# total number of paid reviews
paid_reviews = vineYes_df.count()
paid_reviews

4781

In [13]:
# total number of 5 star reviews
five_star_count = helpful_df.filter(col('star_rating')==5).count()
five_star_count

170404

In [14]:
# number of 5 star reviews unpaid
unpaid_reviews_five_star = vineNo_df.filter(col('star_rating')==5).count()
unpaid_reviews_five_star

168800

In [15]:
# number of 5 star reviews paid
paid_reviews_five_star = vineYes_df.filter(col('star_rating')==5).count()
paid_reviews_five_star

1604

In [16]:
# percent of unpaid five star reviews in all unpaid reviews
unpaid_and_five_prct = (unpaid_reviews_five_star / unpaid_reviews) *100
unpaid_and_five_prct

50.782954015553784

In [17]:
# percent of paid five star reviews in all paid reviews
paid_and_five_prct = (paid_reviews_five_star / paid_reviews) *100
paid_and_five_prct

33.5494666387785

other analysis

In [ ]:
# percent of unpaid 5 star reviews in number of total five star reviews
prct_unpaid = (unpaid_reviews_five_star / five_star_count) *100
prct_unpaid

In [ ]:
# percent of 5 paid star reviews in number of total five star reviews
prct_paid = (paid_reviews_five_star / five_star_count) *100
prct_paid

In [ ]:
# percent of unpaid 5 star in total_reviews
prct_unpaidfsr_total = (unpaid_reviews_five_star / total_reviews) *100
prct_unpaidfsr_total

In [ ]:
# percent of paid 5 star in total_reviews
prct_paidfsr_total = (paid_reviews_five_star / total_reviews) *100
prct_paidfsr_total